In [11]:
# ------------------------------------------------------------
# 1️⃣ 문서 로드 및 분할
# ------------------------------------------------------------
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Word(docx) 문서를 읽어오는 로더
loader = Docx2txtLoader("./tax.docx")

# 긴 문서를 LLM이 다루기 쉽도록 chunk로 쪼갠다.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,   # 한 조각(청크) 최대 길이(문자 기반; 토큰 유사)
    chunk_overlap=200, # 앞/뒤 청크끼리 겹쳐줄 길이 (문맥 보존)
)

# load_and_split로 바로 쪼개진 Document 리스트를 얻는다.
document_list = loader.load_and_split(text_splitter=text_splitter)

In [12]:
# ------------------------------------------------------------
# 2️⃣ 임베딩 & Pinecone에 벡터로 저장
# ------------------------------------------------------------
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone
from langchain_community.vectorstores import Pinecone as PineconeVectorStore

load_dotenv()  # .env에서 OPENAI_API_KEY, PINECONE_API_KEY 읽어옴

# 텍스트 → 벡터로 바꿔줄 임베딩 모델
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

# Pinecone 초기화
index_name = "tax-index"
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

# 문서들을 벡터화해서 Pinecone 인덱스에 넣는다.
# 주의: from_documents()의 첫 번째 파라미터는 documents= 여야 한다.
database = PineconeVectorStore.from_documents(
    documents=document_list,   # ✅ 정확한 이름
    embedding=embedding,
    index_name=index_name,
)

In [17]:
# ------------------------------------------------------------
# 3️⃣ 질문 & 유사 문서 검색 (retrieval)
# ------------------------------------------------------------
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"

# 질문과 가장 가까운 문서 chunk k개를 벡터DB에서 가져온다.
retrieved_docs = database.similarity_search(query, k=5)

# LLM이 이해할 수 있도록, 문서 내용만 꺼내 하나의 큰 문자열로 합친다.
context_text = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [18]:
# ------------------------------------------------------------
# 4️⃣ 프프롬프트 템플릿 + LLM → "체인" 만들기
# ------------------------------------------------------------
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# 실제로 답변을 생성할 LLM (대화형)
llm = ChatOpenAI(model="gpt-4o")

# PromptTemplate: LLM에 보낼 "서식"을 미리 만들어 둔다.
# {context}, {question} 자리에 나중에 실제 값이 들어감.
rag_prompt_template = PromptTemplate(
    template="""
    You are a helpful assistant that can answer questions about Korean tax law.
    Use the following context to answer **accurately and concisely in Korean language**.

    Context:
    {context}

    Question:
    {question}

    Answer:
    """,
    input_variables=["context", "question"]
)

# ---- 체인(chain)이 뭐냐? ----
# rag_prompt_template | llm  는 “프롬프트를 만든 다음 그 결과를 LLM에 바로 넣어라”는 **연결**을 뜻함.
# 즉, PromptTemplate의 출력(완성된 문자열) → LLM의 입력 으로 곧장 넘어가는 파이프.
rag_chain = rag_prompt_template | llm
rag_chain

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n    You are a helpful assistant that can answer questions about Korean tax law.\n    Use the following context to answer **accurately and concisely in Korean language**.\n\n    Context:\n    {context}\n\n    Question:\n    {question}\n\n    Answer:\n    ')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x144cd00a0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x144cd3010>, root_client=<openai.OpenAI object at 0x144ca7550>, root_async_client=<openai.AsyncOpenAI object at 0x144cd28f0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [19]:
# ------------------------------------------------------------
# 5️⃣ 체인 실행: invoke
# ------------------------------------------------------------
# invoke는 “이 체인을 여기 입력값으로 한 번 실행해”라는 의미.
# dict를 넣으면 PromptTemplate가 {context}, {question}를 채우고,
# 그 결과 문자열을 LLM이 받아 답변을 생성한다.
ai_message = rag_chain.invoke({
    "context": context_text,
    "question": query
})
ai_message

AIMessage(content='소득세 계산은 여러 요인에 따라 다를 수 있습니다. 예를 들어, 기본공제, 인적공제, 세액 공제, 기타 소득공제 등이 포함된 후에 최종 소득세가 산출됩니다. 그러나 귀하의 정보 제공에 따라 단순화된 예시로 대략적인 소득세를 계산해 보겠습니다. 5천만 원의 연봉에 대해 기본적인 세율을 적용하면, 소득세 공식에 따라 대략적인 금액을 산출할 수 있습니다. 한국의 소득세율은 구간별로 달라지며, 정확한 계산을 위해서는 각자의 세무 전문가와 상담하는 것이 가장 좋습니다. 대략적인 참고용으로만 생각하시고, 세부 사항은 전문가와 상의하시기 바랍니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 174, 'prompt_tokens': 4674, 'total_tokens': 4848, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 2688}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_65564d8ba5', 'id': 'chatcmpl-CWMkOOvvFZ4x7NpTYTA5lXgQeRRrC', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--60120a84-920e-441f-ae4d-1c7a625225a1-0', usage_metadata={'input_tokens': 4674, 'output_toke

In [20]:
# ------------------------------------------------------------
# 6) 결과 출력 (질문/컨텍스트/답변)
# ------------------------------------------------------------
print("🧩 질문:")
print(query)

print("\n📚 검색된 컨텍스트:")
# 너무 길면 앞부분만 보자. 전부 보고 싶으면 그냥 context_text 출력.
print(context_text.replace("\n", "")[:1200], "...\n(생략)")

print("\n💬 답변:")
print(ai_message.content)

# 참고: retrieved_docs[i].metadata 로 파일 경로, 페이지 등 메타데이터도 볼 수 있음

🧩 질문:
연봉 5천만원인 직장인의 소득세는 얼마인가요?

📚 검색된 컨텍스트:
[전문개정 2009. 12. 31.]제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.[전문개정 2009. 12. 31.]제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.[전문개정 2009. 12. 31.]제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>제1절 비과세 <개정 2009. 12. 31.>제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31., 2025. 10. 1.>1. 「공익신탁법」에 따른 공익신탁의 이익2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발

개념 설명 (체인, invoke를 이해하기 쉽게)
1) 체인(chain) = 연결된 단계들
“이 단계 결과를 다음 단계 입력으로 자동 전달해줘”
LangChain 최신 문법에선 파이프(|) 연산자로 연결해요.
rag_prompt_template | llm
→ “프롬프트(문자열) 만들어 → LLM에게 바로 줘서 답을 받아라”는 작업 파이프라인.
2) invoke = 한 번 실행
rag_chain.invoke({...})
→ 이 파이프라인을 주어진 입력으로 1회 실행.
비유: “빵 반죽(입력)을 넣으면, 체인(파이프라인)이 자동으로 반죽 → 발효 → 굽기 단계를 지나 **완성 빵(출력)**을 내놓는다.”
3) 자주 나오는 메서드 3형제
invoke(input_dict) : 단건 실행
batch([input_dict1, input_dict2, ...]) : 여러 건 일괄 실행
ainvoke(input_dict) : 비동기 한 건 실행 (await 필요)